In [14]:
from uszipcode import SearchEngine
import networkx as nx
import pickle


search = SearchEngine(simple_zipcode=False, db_file_dir="./zip_code_cache")

In [15]:
def get_zip_data(lat, lon):
    zip_code_data = search.by_coordinates(lat, lon, returns=1)[0]
    return {"zipcode": zip_code_data.zipcode,
            "population": zip_code_data.population} 

In [16]:
get_zip_data(42.3,-71.1)

{'zipcode': '02121', 'population': 25978}

In [17]:
get_zip_data(42.3,-72.1)

{'zipcode': '01531', 'population': 999}

In [31]:

from vars import DATASET
print(DATASET)

g = nx.read_gpickle("02_" + "cambridge" + ".gpickle")

cambridge-small


In [32]:
nodes_data_subset = g.nodes().data()


In [33]:
len(nodes_data_subset)

7202

In [ ]:
count = 0
with open('zip_code_dict.pickle', 'rb') as handle:
    zip_code_dict = pickle.load(handle)

for node_data in nodes_data_subset:
    count += 1
    print(count) 
    node_id = node_data[1]["id"]
    lon,lat = node_data[1]['lon'], node_data[1]['lat']
    if node_id not in zip_code_dict:
        zip_code_dict[node_id] = get_zip_data(lat, lon)
    if count % 100 == 0: 
        with open('zip_code_dict.pickle', 'wb') as handle:
            pickle.dump(zip_code_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    
print("done")

with open('zip_code_dict.pickle', 'wb') as handle:
    pickle.dump(zip_code_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
nodes_data_subset

In [25]:
with open('zip_code_dict.pickle', 'rb') as handle:
    b = pickle.load(handle)   
    
print(len(b))

{'61322871': {'zipcode': '02139', 'population': 36349}, '621117424': {'zipcode': '02139', 'population': 36349}, '61322877': {'zipcode': '02139', 'population': 36349}, '61327028': {'zipcode': '02139', 'population': 36349}, '621117407': {'zipcode': '02139', 'population': 36349}, '61326985': {'zipcode': '02139', 'population': 36349}, '61328091': {'zipcode': '02139', 'population': 36349}, '61327098': {'zipcode': '02139', 'population': 36349}, '61317553': {'zipcode': '02139', 'population': 36349}, '61328093': {'zipcode': '02139', 'population': 36349}, '61326974': {'zipcode': '02139', 'population': 36349}, '61327071': {'zipcode': '02139', 'population': 36349}, '61322898': {'zipcode': '02139', 'population': 36349}, '61322880': {'zipcode': '02139', 'population': 36349}, '61322874': {'zipcode': '02139', 'population': 36349}, '61328038': {'zipcode': '02139', 'population': 36349}, '61328115': {'zipcode': '02139', 'population': 36349}, '61328117': {'zipcode': '02139', 'population': 36349}, '613317

34

AttributeError: 'dict' object has no attribute 'haskey'